## Create a list of character names of the books and a word co-occurence matrix

In [21]:
import pandas as pd
import numpy as np 
import sklearn
import seaborn as sns
import pandas as pd
from pathlib import Path
import os
import sys
import csv
import numpy as np

import nltk                                                             # Natural language Toolkit
from nltk.stem import SnowballStemmer                                   # Porter's II Stemmer
from nltk import word_tokenize                                          # Document tokenizer
from nltk.corpus import stopwords  
#nltk.download('wordnet')
#nltk.download('omw-1.4')

from nltk.tokenize import word_tokenize 
import nltk
from nltk.stem import WordNetLemmatizer

from nltk.tokenize import RegexpTokenizer

In [119]:
from pandas.core.common import flatten

In [4]:
raw = pd.read_csv('text_mining_termpaper/data/corpus_children.csv')

In [5]:
raw.shape

(1057, 5)

## Minimalistic preprocessing

In [6]:
df=raw.copy()

In [9]:
# preprocess text
df['clean_text']=df['text'].str.replace(r"\n", " ")
df['clean_text']=df['clean_text'].str.replace(r"\r", "")
# remove extra spaces
df['clean_text']=df['clean_text'].str.replace(r"  ", " ")

# lowercase
df['clean_text']=df['clean_text'].str.lower()

/var/folders/hc/qzy2rzvj5cg38v_607m73p640000gn/T/ipykernel_54056/3662954077.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_text']=df['text'].str.replace(r"\n", " ")
/var/folders/hc/qzy2rzvj5cg38v_607m73p640000gn/T/ipykernel_54056/3662954077.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_text']=df['clean_text'].str.replace(r"\r", "")


In [10]:
stopwords_dict = {word: 1 for word in stopwords_wo_pronoun}
df["clean_text2"]=""
for i in range(len(df)): 
    text = " ".join([word for word in df["clean_text"][i].split() if word not in stopwords_dict])
    df["clean_text2"][i]=text

/var/folders/hc/qzy2rzvj5cg38v_607m73p640000gn/T/ipykernel_54056/334731023.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["clean_text2"][i]=text


In [11]:
# remove \ from text
import re
df["clean_text2"]=[x.replace(r'_',"") for x in df["clean_text2"]]
df["clean_text2"]=[x.replace(r'*',"") for x in df["clean_text2"]]
# remove text with capitalized words
df["clean_text2"]=[re.sub(r'\b[A-Z]+\b', "", x) for x in df["clean_text2"]]

In [12]:
df["clean_text2"]=[x.replace("\\'","'") for x in df["clean_text2"]]
df["clean_text2"]=[x.replace("chapter","") for x in df["clean_text2"]]

In [13]:
# remove Roman numbers
pattern = r"\b(?=[mdclxviΙ])m{0,4}(cm|cd|d?c{0,3})(xc|xl|l?x{0,3})(iΙx|[iΙ]v|v?[iΙ]{0,3})\b\.?"
df["clean_text2"]=[re.sub(pattern, '', text) for text in df["clean_text2"]]

In [14]:
print(df['clean_text2'][0])

[illustration] alice's adventures wonderland lewis carroll millennium fulcrum edition 3.0 contents   rabbit-hole   pool tears   caucus-race long tale   rabbit sends little bill   advice caterpillar   pig pepper   mad tea-party   queen's croquet-ground  ix. mock turtle's story   lobster quadrille   stole tarts?   alice's evidence   rabbit-hole alice beginning get tired sitting her sister bank, nothing do: twice she peeped book her sister reading, pictures conversations it, "and use book," thought alice "without pictures conversations?" she considering her mind (as well she could, hot day made her feel sleepy stupid), whether pleasure making daisy-chain would worth trouble getting picking daisies, suddenly white rabbit pink eyes ran close her. nothing very remarkable that; alice think very much way hear rabbit say itself, "oh dear! oh dear! shall late!" (when she thought afterwards, occurred her she ought wondered this, time seemed quite natural); rabbit actually took watch waistcoat-poc

In [16]:
# export preprocessed df
df.to_csv("preprocessed_corpus.csv")

## Extract character names

In [3]:
project_dir="/Users/pazma/Documents/BSE/text_mining/term_project/"
os.chdir(project_dir)

In [123]:
# create a list of the authors and remove them from the text

author_list=list(dict.fromkeys(df["author"].to_list()))
print(author_list[:10])

# there is a weird float nan object in the list, remove it
author_list = [x for x in author_list if x == x]



new_author_list=[]
for i in range(len(author_list)):
     
     
    if (len(author_list[i].split(","))==1):
        first=author_list[i].split(",")[0]
        new_author_list.append(first)
    else:
        
        first=author_list[i].split(",")[1]
        second=author_list[i].split(",")[0]
        new=first + " " + second
        new_author_list.append(new)
    


# remove leading spaces
new_author_list=[x.strip() for x in new_author_list]

# put into lowercase
new_author_list=[x.lower() for x in new_author_list]

print(new_author_list[:10])

['Carroll, Lewis', 'Montgomery, L. M. (Lucy Maud)', 'Burnett, Frances Hodgson', 'Alcott, Louisa May', 'MacDonald, George', 'Alger, Horatio', 'Optic, Oliver', 'Craik, Dinah Maria Mulock', 'Appleton, Victor', 'Lofting, Hugh']
['lewis carroll', 'l. m. (lucy maud) montgomery', 'frances hodgson burnett', 'louisa may alcott', 'george macdonald', 'horatio alger', 'oliver optic', 'dinah maria mulock craik', 'victor appleton', 'hugh lofting']


In [124]:
# remove author names from the text
for i in range(len(new_author_list)): 
    df["clean_text2"]=df["clean_text2"].replace(new_author_list[i], "")

In [125]:
df["clean_text2"][0]

'[illustration] alice\'s adventures wonderland lewis carroll millennium fulcrum edition 3.0 contents   rabbit-hole   pool tears   caucus-race long tale   rabbit sends little bill   advice caterpillar   pig pepper   mad tea-party   queen\'s croquet-ground  ix. mock turtle\'s story   lobster quadrille   stole tarts?   alice\'s evidence   rabbit-hole alice beginning get tired sitting her sister bank, nothing do: twice she peeped book her sister reading, pictures conversations it, "and use book," thought alice "without pictures conversations?" she considering her mind (as well she could, hot day made her feel sleepy stupid), whether pleasure making daisy-chain would worth trouble getting picking daisies, suddenly white rabbit pink eyes ran close her. nothing very remarkable that; alice think very much way hear rabbit say itself, "oh dear! oh dear! shall late!" (when she thought afterwards, occurred her she ought wondered this, time seemed quite natural); rabbit actually took watch waistcoa

### List of male and female names

NLTK Names package, extract male and female names from texts

In [35]:
# nltk.download('names')
from nltk.corpus import names
male_names = [name for name in names.words('male.txt')]
female_names = [name for name in names.words('female.txt')]
names = male_names+female_names

In [17]:
# tokenize text
df['tokens'] = df['clean_text2'].apply(nltk.word_tokenize)

In [ ]:
# df["tokens"][0]

In [59]:
# create a full list from the tokens
word_tokens=list(df['tokens'])

word_tokens=list(flatten(word_tokens))

# remove stopwords
word_tokens=[w.lower() for w in word_tokens]
filtered_sentence = [] 
  
for w in word_tokens: 
    if w not in stopwords: 
        filtered_sentence.append(w) 
print(filtered_sentence[3]) 

In [122]:
print(filtered_sentence[:30])

['[', 'illustration', ']', 'alice', "'s", 'adventures', 'wonderland', 'lewis', 'carroll', '3.0', 'contents', '.', 'rabbit-hole', '.', 'pool', 'tears', '.', 'caucus-race', 'long', 'tale', '.', 'rabbit', 'sends', 'little', 'bill', '.', 'advice', 'caterpillar', '.', 'pig']


In [76]:
# lemmatize words
# Lowercase and lemmatise 
lemmatiser = WordNetLemmatizer()
lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in filtered_sentence]

In [77]:
print(lemmas[:30])

['[', 'illustration', ']', 'alice', "'s", 'adventure', 'wonderland', 'lewis', 'carroll', '3.0', 'content', '.', 'rabbit-hole', '.', 'pool', 'tear', '.', 'caucus-race', 'long', 'tale', '.', 'rabbit', 'send', 'little', 'bill', '.', 'advice', 'caterpillar', '.', 'pig']


Filter for female and male names and add these to the list:
- Mr,Mrs,Ms 
- king,queen 
- mother,father,daughter,son,aunt,niece,uncle, wife, husband 

In [73]:
female_names=female_names+["Mrs", "Ms", "Queen", "Mother", "Daughter", "Aunt", "Niece", "Grandmother", "Wife"]
male_names=male_names+["Mr", "King", "Father", "Son", "Uncle", "Husband", "Grandfather", "Duchess"]

In [78]:
# put names into lower case
female_names_lower=[name.lower() for name in female_names]
male_names_lower=[name.lower() for name in male_names]

In [7]:
stopwords = nltk.corpus.stopwords.words('english')
remove_pronouns=['he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself']
# stopwords = [word for word in remove_pronouns if word not in stopwords]

In [8]:
stopwords_wo_pronoun=list(set(stopwords)-set(remove_pronouns))

In [80]:
# extract female and male names
all_male_names=[]
all_female_names=[]
# all_other=[]

for i in range(len(lemmas)):
 
    # if a character name appears in the lemmas list then add it to the book female/male list    
        if (lemmas[i] in male_names_lower)==True:
            all_male_names.append(lemmas[i])
        elif (lemmas[i] in female_names_lower)==True:
            all_female_names.append(lemmas[i])

In [89]:
len(all_female_names)

666577

In [97]:
female_character_names=list(set(all_female_names))
male_character_names=list(set(all_male_names))

In [99]:
print("Number of unique female characters in the books: ", len(female_character_names))
print("Number of unique male characters in the books: ",len(male_character_names))

(1445, 1899)

In [144]:
print(female_character_names[:30])

['fulvia', 'gwen', 'marjie', 'robby', 'jessica', 'gwendolin', 'bernardina', 'caron', 'pierette', 'fleur', 'berry', 'briny', 'jocelin', 'tansy', 'arden', 'winna', 'viola', 'marcella', 'ursuline', 'jolie', 'clementina', 'terra', 'adrienne', 'gertie', 'bell', 'susannah', 'gypsy', 'anny', 'ethelind', 'zahara']


In [145]:
print(male_character_names[:30])

['shea', 'austen', 'griff', 'abram', 'bennet', 'rudolph', 'darrell', 'osmond', 'michele', 'christopher', 'chad', 'ambrose', 'damon', 'webster', 'verney', 'chauncey', 'morton', 'reagan', 'shaun', 'lancelot', 'clem', 'vladimir', 'josephus', 'rudd', 'ely', 'stanton', 'wally', 'hart', 'town', 'leonidas']


In [100]:
# export to txt
with open('female_character_names.txt', 'w') as f:
    for item in female_character_names:
        f.write("%s\n" % item)
        
with open('male_character_names.txt', 'w') as f:
    for item in male_character_names:
        f.write("%s\n" % item)

In [18]:
# export cleaned df
df.to_csv("preprocessed_corpus.csv")